<a href="https://colab.research.google.com/github/gglchrm/tarot_NN/blob/main/GPT_introduction_HABR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers
!pip install accelerate
!pip install torch torchdata transformers datasets loralib peft pandas numpy

In [ ]:
# Restart session here

In [1]:
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments
from torch.optim import Adam
import pandas as pd
import torch
device = torch.device("cuda:0") # cpu or torch.device("cuda:0")

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_name = "google/flan-t5-base" # Base model to use
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

#from transformers import LlamaTokenizer, LlamaForCausalLM
#model_name = "openlm-research/open_llama_3b_v2" # Base model to use
#tokenizer = LlamaTokenizer.from_pretrained(model_name)
#model = LlamaForCausalLM.from_pretrained(model_name).to(device)

training_file = "tarot_readings.csv.1" # CSV file to use
num_epochs = 3 # Number of iterations to train
num_rows = 500 # Number of rows to use for training

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when poss

In [2]:
def create_tarot_dataset(csv_file, tokenizer, num_rows=None):
    data = pd.read_csv(csv_file)

    if num_rows:
        data = data[:num_rows]

    def tokenize(row):
        prompt = "Give me a one paragraph tarot reading if I pull the cards {}, {} and {}.".format(row['Card 1'], row[' Card 2'], row[' Card 3'])
        reading = row[' Reading']

        inputs = tokenizer.encode_plus(prompt, add_special_tokens=True, padding='max_length', max_length=128, truncation=True, return_tensors='pt')
        target = tokenizer.encode_plus(reading, add_special_tokens=True, padding='max_length', max_length=128, truncation=True, return_tensors='pt')

        return {'input_ids': inputs['input_ids'].squeeze(), 'attention_mask': inputs['attention_mask'].squeeze(), 'target_ids': target['input_ids'].squeeze(), 'target_attention_mask': target['attention_mask'].squeeze()}

    dataset = data.apply(tokenize, axis=1).tolist()

    return dataset

In [3]:
from transformers import Trainer, TrainingArguments

def fine_tune_model(model, optimizer, batch, device):
    training_args = TrainingArguments(
    output_dir="./finetuned", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    warmup_steps=10,# number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=8, # to make "virtual" batch size larger
    )

    trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=batch,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)


    trainer.train()


    #model.train()
    #input_ids = batch['input_ids'].to(device)
    #attention_mask = batch['attention_mask'].to(device)
    #labels = batch['target_ids'].to(device)
    #decoder_attention_mask = batch['target_attention_mask'].to(device)

    #outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)
    #loss = outputs.loss
    #optimizer.zero_grad()
    #loss.backward()
    #optimizer.step()

    #return loss.item()

In [4]:
def tacot_reading(model, tokenizer, card1, card2, card3):
    prompt = "Give me a one paragraph tarot reading if I pull the cards {}, {} and {}.".format(card1, card2, card3)

    inputs = tokenizer(prompt, return_tensors="pt")
    completion = tokenizer.decode(model.generate(inputs["input_ids"], max_new_tokens=1000)[0], skip_special_tokens=True)

    print("Prompt: {}".format(prompt))
    print("Response: {}".format(completion))
    print()

    return completion

In [5]:
print("* Running 3 inferences (pre-training)...")
tacot_reading(model, tokenizer, "The moon", "Two of Swords", "Three of Wands")
tacot_reading(model, tokenizer, "The hermit", "Ace of Pentacles", "Judgement")
tacot_reading(model, tokenizer, "Seven of Cups", "The chariot", "King of Swords")

* Running 3 inferences (pre-training)...
Prompt: Give me a one paragraph tarot reading if I pull the cards The moon, Two of Swords and Three of Wands.
Response: Two of Swords

Prompt: Give me a one paragraph tarot reading if I pull the cards The hermit, Ace of Pentacles and Judgement.
Response: The hermit, Ace of Pentacles and Judgement

Prompt: Give me a one paragraph tarot reading if I pull the cards Seven of Cups, The chariot and King of Swords.
Response: Seven of Cups, The chariot and King of Swords



'Seven of Cups, The chariot and King of Swords'

In [6]:
print("* Creating dataset from [{}]...".format(training_file))
dataset = create_tarot_dataset(training_file, tokenizer, num_rows)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True)

* Creating dataset from [tarot_readings.csv.1]...


In [9]:
print("* Training model for {} epochs..".format(num_epochs))
optimizer = torch.optim.AdamW(model.parameters(),lr=1e-5),None
for epoch in range(num_epochs):
    loss = 0
    for batch in data_loader:
        loss += fine_tune_model(model, optimizer, batch, device)
    print("Epoch {} average loss: {}".format((epoch+1), (loss / len(data_loader))))

* Training model for 3 epochs..


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
print("* Running 3 inferences (post-training)...")
tacot_reading(model, tokenizer, "The moon", "Two of Swords", "Three of Wands")
tacot_reading(model, tokenizer, "The hermit", "Ace of Pentacles", "Judgement")
tacot_reading(model, tokenizer, "Seven of Cups", "The chariot", "King of Swords")